In [ ]:
# Datos: http://saludata.saludcapital.gov.co/osb/datos_abiertos_osb/enf-transmisibles/OSB_EnfTransm-COVID-19.csv

In [ ]:
import requests
import pandas as pd
#import geopandas as gpd
from google.cloud import storage

In [ ]:
URL = 'http://saludata.saludcapital.gov.co/osb/datos_abiertos_osb/enf-transmisibles/OSB_EnfTransm-COVID-19.csv'
FILEPATH = './data/covid19_bogota_raw.csv'

In [ ]:
r = requests.get( URL, stream = True )

with open( FILEPATH, 'wb' ) as f:
    f.write( r.content )

In [ ]:
# Loading the data
covid_bog = pd.read_csv( FILEPATH, encoding = 'ISO-8859-1', sep = ';', skiprows = 4, skipfooter = 4 )

In [ ]:
covid_bog.columns

In [ ]:
covid_bog.shape

In [ ]:
covid_bog[ 'Fecha de Inicio de Síntomas' ] = pd.to_datetime( covid_bog[ 'Fecha de Inicio de Síntomas' ], format = '%d/%m/%Y', errors = 'coerce' )

In [ ]:
covid_bog[ 'Fecha de diagnóstico' ] = pd.to_datetime( covid_bog[ 'Fecha de diagnóstico' ], format = '%d/%m/%Y' )

In [ ]:
covid_bog[ 'Localidad de residencia' ].replace( { 'Santa Fe': 'Santa Fé' }, inplace = True )

In [ ]:
covid_bog[ 'Atencion' ] = covid_bog[ 'Ubicación' ].str.strip()
del covid_bog[ 'Ubicación' ]

In [ ]:
covid_bog[ 'Estado' ] = covid_bog[ 'Estado' ].str.strip()

In [ ]:
covid_bog['Tipo de caso']= covid_bog[ 'Tipo de caso' ].str.strip(" ")

In [ ]:
covid_bog.dtypes

In [ ]:
def get_age_group( x ):
    if x <= 19:
        return '0 - 19'
    elif x > 19 and x <= 39:
        return '20 - 39'
    elif x > 39 and x <= 59:
        return '40 - 59'
    elif x > 59 and x <= 79:
        return '60 - 79'
    elif x > 79:
          return '80+'
    else:
        return None

covid_bog[ 'Grupo de edad' ] = covid_bog[ 'Edad' ].apply( get_age_group )

In [ ]:
covid_bog.replace( { 'Fallecido (No aplica, No causa Directa)': 'Fallecido', 'Fallecido(No aplica': 'Fallecido', 'fallecido': 'Fallecido', 'hospital': 'Hospital', 'leve': 'Leve' }, inplace = True )

In [ ]:
covid_bog.rename( columns = { 'ID de caso': 'ID_caso', 'Fecha de Inicio de Síntomas': 'Fecha_inicio_sintomas', 'Fecha de diagnóstico': 'Fecha_diagnostico', 'Ciudad de residencia': 'Ciudad', 'Localidad de residencia': 'Localidad', 'Tipo de caso': 'Tipo', 'Ubicación': 'Atencion', 'Grupo de edad': 'Grupo_de_edad' }, inplace = True )

In [ ]:
covid_bog = covid_bog[ [ 'ID_caso', 'Fecha_inicio_sintomas', 'Fecha_diagnostico', 'Ciudad', 'Localidad', 'Edad', 'Sexo', 'Tipo', 'Atencion', 'Estado', 'Grupo_de_edad' ] ]

In [ ]:
covid_bog.dtypes

In [ ]:
covid_bog.tail()

In [ ]:
covid_bog[ 'Fecha_inicio_sintomas' ].hist()

In [ ]:
covid_bog[ 'Fecha_diagnostico' ].hist()

In [ ]:
covid_bog[ 'Atencion' ].unique()

In [ ]:
covid_bog[ 'Estado' ].unique()

In [ ]:
#covid_bog.to_csv( './data/covid19_bogota.csv', index = False )

In [ ]:
#DataFrame to CSV
csv_data=covid_bog.to_csv(index=False)

#Client, bucket and blob to acces gcloud
client= storage.Client()
bucket= client.get_bucket('siscovid')
blob=bucket.blob('covid19_bogota.csv')

#Uploads the file to gcloud
blob.upload_from_string(data=csv_data, content_type='text/csv')